In [1]:
import os
import random

import numpy as np
import argparse
import torch
import cv2

from torchvision import models

import config


In [2]:
def preprocess_image(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # swap the color channels, BGR to RGB
    image = cv2.resize(image, (config.IMAGE_SIZE, config.IMAGE_SIZE)) # resize
    image = image.astype("float32") / 255.0 # scale to [0, 1]
    
    #transform = transforms.Compose([            
    #    transforms.Resize(256),                    
    #    transforms.CenterCrop(224),                
    #    transforms.ToTensor(),                     
    #    transforms.Normalize(                      
    #    mean=[0.485, 0.456, 0.406],                
    #    std=[0.229, 0.224, 0.225]                  
    #)])
    
    image -= config.MEAN # scale with the mean and std in the config file
    image /= config.STD
    image = np.transpose(image, (2, 0, 1)) # set "channels first" ordering
    image = np.expand_dims(image, 0) # add a batch dimension
    
    return image

In [3]:
# define a dictionary that maps model names to their classes
# inside torchvision

MODELS = {
    "vgg16": models.vgg16(pretrained=True),
    "vgg19": models.vgg19(pretrained=True),
    "inception": models.inception_v3(pretrained=True),
    "densenet": models.densenet121(pretrained=True),
    "resnet": models.resnet50(pretrained=True)
}



In [12]:
# load our the network weights from disk, flash it to the current
# device, and set it to evaluation mode
model_idx = random.randint(0, len(MODELS) - 1)
model_name = list(MODELS.keys())[model_idx]

print("[INFO] loading {}...".format(model_name))
model = MODELS[model_name].to(config.DEVICE)
model.eval()

# load the image from disk, clone it (so we can draw on it later),
# and preprocess it
print("[INFO] loading image...")
all_images = os.listdir("images")
img_idx = random.randint(0, len(all_images) - 1)
image = cv2.imread(os.path.join("images", all_images[img_idx]))
orig = image.copy()
image = preprocess_image(image)

# convert the preprocessed image to a torch tensor and flash it to
# the current device
image = torch.from_numpy(image)
image = image.to(config.DEVICE)

# load the preprocessed the ImageNet labels
print("[INFO] loading ImageNet labels...")
imagenetLabels = dict(enumerate(open(config.IN_LABELS)))

[INFO] loading densenet...
[INFO] loading image...
[INFO] loading ImageNet labels...


In [13]:
# classify the image and extract the predictions
print("[INFO] classifying image with '{}'...".format(model_name))
logits = model(image)
probabilities = torch.nn.Softmax(dim=-1)(logits)
sortedProba = torch.argsort(probabilities, dim=-1, descending=True)

# loop over the predictions and display the rank-5 predictions and
# corresponding probabilities to our terminal
for (i, idx) in enumerate(sortedProba[0, :5]):
    print("{}. {}: {:.2f}%".format
        (i, imagenetLabels[idx.item()].strip(),
        probabilities[0, idx.item()] * 100))

[INFO] classifying image with 'densenet'...
0. daisy: 26.96%
1. bee: 15.33%
2. sulphur_butterfly, sulfur_butterfly: 12.98%
3. hummingbird: 11.99%
4. sea_slug, nudibranch: 5.29%


In [14]:
# draw the top prediction on the image and display the image to
# our screen
(label, prob) = (imagenetLabels[probabilities.argmax().item()],
    probabilities.max().item())

cv2.putText(orig, "Label: {}, {:.2f}%, predicted using {}".format(label.strip(), prob * 100, model_name),
    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

cv2.imshow("Classification", orig)
cv2.waitKey(0)

-1